# original paper code

In [ ]:
%pip install tensorflow[and-cuda]
%pip install -q -U keras-tuner


In [ ]:
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime
import pandas as pd

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
IMPORT_COUNT = 1990000
TEST_COUNT = 10000

In [ ]:
# Generate random seed
#myrand=np.random.randint(1, 99999 + 1)
myrand=71926
np.random.seed(myrand)
tf.random.set_seed(myrand)
print("Random seed is:",myrand)

In [ ]:
PREVIOUS_TIMESTEP_COUNT = 4
TOTAL_DATA_NUM = IMPORT_COUNT-PREVIOUS_TIMESTEP_COUNT

In [ ]:
# convert the sequence of generated numbers to 4 inputs and one output
def strided(a, L):
	shp = a.shape
	s  = a.strides
	nd0 = shp[0]-L+1
	shp_in = (nd0,L)+shp[1:]
	strd_in = (s[0],) + s
	return np.lib.stride_tricks.as_strided(a, shape=shp_in, strides=strd_in)

In [ ]:
RNG_OUTPUT_FILENAME="xorshift128.txt"
df = np.genfromtxt(RNG_OUTPUT_FILENAME,delimiter='\n',dtype='uint64')[:IMPORT_COUNT]

In [ ]:
# calculates how many bits are in the output.
BIT_WIDTH = np.ceil(np.log2(np.amax(df))).astype(int)

In [ ]:
# convert the generated numbers to binary sequences
df_as_bits =(df[:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int)
df_as_frames = strided(df_as_bits, PREVIOUS_TIMESTEP_COUNT+1)

In [ ]:
indicies = np.arange(TOTAL_DATA_NUM,dtype='uint64')
np.random.shuffle(indicies)
df_as_frames=df_as_frames[indicies]

In [ ]:
# convert the data into inputs and outputs
y = df_as_frames[:,-1,:]
X = df_as_frames[:,:-1,]
X = X.reshape([X.shape[0], X.shape[1]*X.shape[2]])

In [ ]:
# Convert the data into train and test data
X_train = X[TEST_COUNT:]
X_test = X[:TEST_COUNT]
y_train = y[TEST_COUNT:]
y_test = y[:TEST_COUNT]

In [ ]:
def build_model(hp):
	LOSS="binary_crossentropy"
	model = Sequential()
	model.add(Dense(1024, activation='relu',input_shape=[X.shape[1]] ))
	model.add(Dense(y.shape[1], activation='sigmoid'))
	opt = keras.optimizers.Nadam(
		learning_rate=hp.Float("learning_rate", 10**(-5), 10**(-3),sampling="log"),
		epsilon=hp.Float("epsilon",1e-7,1e-5,sampling="log"),
		beta_1=hp.Float("beta_1",.8,.9,sampling="reverse_log"),
		beta_2=hp.Float("beta_2",.8,.9,sampling="reverse_log"),
		)
	model.compile(optimizer=opt, loss=LOSS,metrics=['binary_accuracy'])
	return model

In [ ]:
#define call back functions
stopEarly = tf.keras.callbacks.EarlyStopping(
	monitor='binary_accuracy', min_delta=.001, patience=3, verbose=0, mode='auto', restore_best_weights=False
)

log_dir = "hyperparameters/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)

In [ ]:
# extract a short set from the training for hyper parameter tuning
X_train_short= X_train[:600000]
y_train_short= y_train[:600000]

In [ ]:
tuner = kt.BayesianOptimization(
    build_model,  # Your model-building function
    objective='binary_accuracy',  # The objective to optimize
    max_trials=20,  # The total number of trials (model configurations) to test
    directory='bayes_opt',  # Directory to store logs
    project_name='bayes',
    seed=myrand  # Optional seed for reproducibility
)

In [ ]:
# %%time
# Tuning the hyper parameters


# tuner.search(X_train_short, y_train_short,batch_size=256, epochs=50, validation_data=(X_test,y_test),callbacks=[stopEarly,tensorboard_callback])
tuner.results_summary()

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
# use the best model for training
model = tuner.hypermodel.build(best_hps)
best_hps.values

In [ ]:
def train_model(model, X = X_train, Y=y_train, epochs=10, batch_size=512,verbose=0, log_dir = "dense_model/"):
    log_dir += datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)
    model.fit(X, Y, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[tensorboard_callback],verbose=verbose)
    return model

In [ ]:
%%time
my_model_trained = train_model(model, epochs=100)

In [ ]:
results = model.evaluate(X_test, y_test, batch_size=256)
print("test loss: %f, test acc: %s" % tuple(results))

In [ ]:
my_model_trained.save("xorshift128_model.h5")